In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf


In [70]:
stocklist = ['MSFT', 'BHP', 'TLS', 'MCD', 'AAXJ', 'XPP']


In [71]:
mc_sims = 50
T=100 #timeframe in days
initial_portfolio = 1000

count = 100
def get_data(stocks, start, end):
    data = yf.download(stocks, start, end)["Close"]
    returns = data.pct_change()
    meanReturn = returns.mean()
    covMatrix = returns.cov()
    return meanReturn, covMatrix

stocks = [stock for stock in stocklist]
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=300)
meanReturns, covMatrix = get_data(stocks, startDate, endDate)

def random_port_size():
  weights = np.random.random(len(meanReturns))
  #ensure that the sum of the weights array is '1'
  weights /= np.sum(weights)
  meanM = np.full(shape=(T, len(weights)), fill_value=meanReturns)
  meanM = meanM.T
  portfolio_sims = np.full(shape=(T, mc_sims), fill_value=0.0)

  for m in range(0, mc_sims):
    Z = np.random.normal(size=(T, len(weights)))
    L = np.linalg.cholesky(covMatrix)
    daily_returns = meanM + np.inner(L, Z)
    portfolio_sims[:, m] = np.cumprod(np.inner(weights, daily_returns.T)+1)*initial_portfolio

  final_portfolio_values = portfolio_sims[-1]

  # Calculate the percentage change from the initial portfolio value
  returns = (final_portfolio_values - initial_portfolio) / initial_portfolio

  def num_calculator(value_in_pct):
      # Count the number of simulations with at least a specified increase
      num_simulations = np.sum(returns >= (value_in_pct / 100)) / len(returns) * 100
      return num_simulations


  return weights, num_calculator(10), num_calculator(20), num_calculator(30), num_calculator(40), num_calculator(50)

[*********************100%%**********************]  6 of 6 completed


In [72]:
best_10 = 0
best_weights_10 = None
best_20 = 0
best_weights_20 = None
best_30 = 0
best_weights_30 = None
best_40 = 0
best_weights_40 = None
best_50 = 0
best_weights_50 = None

for i in range(count):
  weighted_df, returns_10, returns_20, returns_30, returns_40, returns_50 = random_port_size()
  if returns_10 > best_10:
    best_10 = returns_10
    best_weights_10 = weighted_df
  if returns_20 > best_20:
    best_20 = returns_20
    best_weights_20 = weighted_df
  if returns_30 > best_30:
    best_30 = returns_30
    best_weights_30 = weighted_df
  if returns_40 > best_40:
    best_40 = returns_40
    best_weights_40 = weighted_df
  if returns_50 > best_50:
    best_50 = returns_50
    best_weights_50 = weighted_df



In [73]:
print('> 10%', best_10)
print(best_weights_10)
print('\n')
print('> 20%', best_20)
print(best_weights_20)
print('\n')
print('> 30%', best_30)
print(best_weights_30)
print('\n')
print('> 40%', best_40)
print(best_weights_40)
print('\n')
print('> 50%', best_50)
print(best_weights_50)


> 10% 76.0
[0.02382909 0.01839577 0.09371584 0.12271317 0.46908994 0.27225619]


> 20% 66.0
[0.02382909 0.01839577 0.09371584 0.12271317 0.46908994 0.27225619]


> 30% 54.0
[0.02382909 0.01839577 0.09371584 0.12271317 0.46908994 0.27225619]


> 40% 36.0
[0.02382909 0.01839577 0.09371584 0.12271317 0.46908994 0.27225619]


> 50% 30.0
[0.02382909 0.01839577 0.09371584 0.12271317 0.46908994 0.27225619]


In [74]:
best_weights_10.shape

(6,)

In [75]:
def get_data(stocks, start, end):
      data = yf.download(stocks, start, end)["Close"]
      returns = data.pct_change()
      meanReturn = returns.mean()
      covMatrix = returns.cov()
      return meanReturn, covMatrix

stocks = [stock for stock in stocklist]
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=300)
meanReturns, covMatrix = get_data(stocks, startDate, endDate)


meanM = np.full(shape=(T, len(meanReturns)), fill_value=meanReturns)
meanM = meanM.T

portfolio_sims = np.full(shape=(T, mc_sims), fill_value=0.0)
for m in range(0, mc_sims):
  Z = np.random.normal(size=(T, len(best_weights_10)))
  L = np.linalg.cholesky(covMatrix)
  daily_returns = meanM + np.inner(L, Z)
  portfolio_sims[:, m] = np.cumprod(np.inner(best_weights_10, daily_returns.T)+1)*initial_portfolio

[*********************100%%**********************]  6 of 6 completed



**include value at risk(vAr) and conditional value at risk(CVAR)**

In [76]:
def mcVar(returns, alpha = 5):
  """
  Input: panda series of returns
  output: percentile on return distribution based on confidence level
  """

  if isinstance(returns, pd.Series):
    return np.percentile(returns, alpha)

  else:
    raise TypeError("Input must be a pandas series")


def mcCVar(returns, alpha = 5):
  """
  Input: panda series of returns
  output: percentile on return distribution based on confidence level
  """

  if isinstance(returns, pd.Series):
    belowVar = returns<= mcVar(returns, alpha = alpha)
    return returns[belowVar].mean()

  else:
    raise TypeError("Input must be a pandas series")


portResult = pd.Series(portfolio_sims[-1, :])

VaR = initial_portfolio - mcVar(portResult, alpha=5)
CVar = initial_portfolio - mcCVar(portResult, alpha=5)


print(f'Value at risk: ${VaR}')
print(f'Conditional value at risk: ${CVar}')


Value at risk: $273.6592950460945
Conditional value at risk: $295.8662091130642


working with alpaca api(paper trading)

In [77]:
apiKey = ''
secretKey = ''
baseUrl = 'https://paper-api.alpaca.markets/'

In [78]:
!pip install alpaca-trade-api
!pip install alpaca-py

In [79]:
stocklist


['MSFT', 'BHP', 'TLS', 'MCD', 'AAXJ', 'XPP']

check market open


In [80]:
import alpaca_trade_api as tradeapi

api = tradeapi.REST(apiKey, secretKey, baseUrl)

# Check if the market is open now.
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))


The market is closed.


Get the stocks available


In [81]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest

trading_client = TradingClient(apiKey, secretKey)

for stocks in stocklist:
  try:
    # search for AAPL
    asset = trading_client.get_asset(stocks)

    if asset.tradable:
        print(f'We can trade {stocks}.')
    else:
        print(f'We cannot trade {stocks}.')

  except:
    print(f'We cannot trade {stocks}.')

We can trade MSFT.
We can trade BHP.
We can trade TLS.
We can trade MCD.
We can trade AAXJ.
We can trade XPP.


In [91]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

trading_client = TradingClient(apiKey, secretKey)




for i in range(len(stocklist)):
  stock = yf.download(stocklist[i], progress=False)


  current_price = stock.iloc[-1]['Close']


  # Get the current price
  stk_qty = 0.9*best_weights_10[i]*initial_portfolio / current_price

  # preparing order data
  market_order_data = MarketOrderRequest(
                        symbol=stocklist[i],
                        qty=stk_qty,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.DAY
                    )
  print('Purchased ' + str(stk_qty) + ' of ' + stocklist[i] +  " costing " + "$" + str(current_price*stk_qty))
  print('/n')

  # Market order
  market_order = trading_client.submit_order(
                order_data=market_order_data
                )

[*********************100%%**********************]  1 of 1 completed


Purchased0.05012311671871268ofMSFTcosting$21.446177705693813


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Purchased0.28344786458047927ofBHPcosting$16.55618972689508
Purchased20.774448279578404ofTLScosting$84.34425882636485


[*********************100%%**********************]  1 of 1 completed


Purchased0.43513595027918056ofMCDcosting$110.44185447801519


[*********************100%%**********************]  1 of 1 completed


Purchased5.901327135795823ofAAXJcosting$422.1809486976595


[*********************100%%**********************]  1 of 1 completed

Purchased15.527919816969167ofXPPcosting$245.03057056537153


In [85]:
current_price

15.779999732971191

In [86]:
stk_qty = 0.9*best_weights_10[i]*initial_portfolio / current_price
stk_qty

17.253244241076853

In [87]:
best_weights_10[i]

0.27225618951707947